<a href="https://colab.research.google.com/github/Shyam-Khokhariya/ML-Python/blob/master/C16_Movie_Review_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install pyprind
import pyprind
import pandas as pd
import numpy as np
import re
from string import punctuation

In [0]:
df=pd.read_csv("movie_data.csv",encoding="utf-8")

In [0]:
review="Hello, everyone . My Self ! @ shyam ?"
text=""
for c in review:
  if c not in punctuation:
    text=text+c
  else:
    text=text+" "
print(text)
text=" ".join(text.split()).lower()
print(text)

sequence_length=200
sequences=np.zeros(sequence_length,dtype=str)
r_arr=np.array(text)
sequences[-len(text):]=text[-sequence_length:]
print(sequences)

Hello  everyone   My Self     shyam  
hello everyone my self shyam
['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 '' '' '' '' 'h' 'h' 'h' 'h' 'h' 'h' 'h' 'h' 'h' 'h' 'h' 'h' 'h' 'h' 'h'
 'h' 'h' 'h' 'h' 'h' 'h' 'h' 'h' 'h' 'h' 'h' 'h' 'h']


In [0]:
from collections import Counter
counts = Counter()
pbar = pyprind.ProgBar(len(df['review']),title='Counting words occurrences')
for i,review in enumerate(df['review']):
  #text = ''.join([c if c not in punctuation else ' '+c+' ' for c in review]).lower()
  text=""
  for c in review:
    if c not in punctuation:
      text=text+c
    else:
      text=text+" "
  text=" ".join(text.split()).lower()
  df.loc[i,'review'] = text
  pbar.update()
  counts.update(text.split())
 ## Create a mapping
 ## Map each unique word to an integer
word_counts = sorted(counts, key=counts.get, reverse=True)
print(word_counts[:5])
word_to_int = {word: ii for ii, word in  enumerate(word_counts, 1)}
mapped_reviews = []
pbar = pyprind.ProgBar(len(df['review']),  title='Map reviews to ints')
for review in df['review']:
  mapped_reviews.append([word_to_int[word] for word in review.split()])
  pbar.update()

Counting words occurrences
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:06:26
Map reviews to ints


['the', 'and', 'a', 'of', 'to']


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


In [0]:
#Defining same length sequences

sequence_length=200
sequences=np.zeros((len(mapped_reviews),sequence_length),dtype=int)
for i,row in enumerate(mapped_reviews):
  review_arr=np.array(row)
  sequences[i,-len(row):]=review_arr[-sequence_length:]


In [0]:
X_train=sequences[:25000,:]
y_train=df.loc[:25000,"sentiment"].values
X_test=sequences[25000:,:]
y_test=df.loc[25000:,"sentiment"].values

In [0]:
np.random.seed(123)

#Function for mini-batch
def create_batch_generator(x,y=None,batch_size=64):
  n_batches=len(x)//batch_size
  x=x[:n_batches*batch_size]
  if y is not None:
    y=y[:n_batches*batch_size]
  for ii in range(0,len(x),batch_size):
    if y is not None:
      yield x[ii:ii+batch_size],y[ii:ii+batch_size]
    else:
      yield x[ii:ii+batch_size]
      

In [0]:

class SentimentRNN(object):
    def __init__(self,n_words,seq_len=200,
                 lstm_size=256,num_layers=1,batch_size=64,
                 learning_rate=0.0001,embed_size=200):
        self.n_words=n_words
        self.seq_len=seq_len
        self.lstm_size=lstm_size            #number of hidden units
        self.num_layers=num_layers
        self.batch_size=batch_size
        self.learning_rate=learning_rate
        self.embed_size=embed_size

        self.g= tf.Graph()
        with self.g.as_default():
            tf.set_random_seed(123)
            self.build()
            self.saver=tf.train.Saver()
            self.init_op=tf.global_variables_initializer()

    def build(self):

        #defining placeholder
        tf_x=tf.placeholder(tf.int32,shape=(self.batch_size,self.seq_len),name="tf_x")
        tf_y=tf.placeholder(tf.float32,shape=(self.batch_size),name="tf_y")
        tf_keepprob=tf.placeholder(tf.float32,name="tf_keepprob")

        #creating embeded layer
        embedding=tf.Variable(tf.random_uniform((self.n_words,self.embed_size),minval=-1,maxval=1),name="embedding")
        embed_x=tf.nn.embedding_lookup(embedding,tf_x,name="embeded_x")

        #Define LSTM cell and stack them together
        cells=tf.contrib.rnn.MultiRNNCell(
            [tf.contrib.rnn.DropoutWrapper(
                tf.contrib.rnn.BasicLSTMCell(self.lstm_size),output_keep_prob=tf_keepprob)
                for i in range(self.num_layers)
            ])
        ##Define Initial State
        self.initial_state=cells.zero_state(self.batch_size,tf.float32)
        print(" << initial state >> ",self.initial_state)

        lstm_output,self.final_state=tf.nn.dynamic_rnn(cells,embed_x,initial_state=self.initial_state)

        #   NOTE: LSTM Output Shape:
        #   [batch_size,max time,cells.output_size]
        print(" << lstm_output >> ",lstm_output)
        print(" << final state >> ",self.final_state)

        logits=tf.layers.dense(inputs=lstm_output[:,-1],
                               units=1,activation=None,name="logits")
        logits=tf.squeeze(logits,name="logits_squeezed")

        print(" << logits >> ",logits)
        y_proba=tf.nn.sigmoid(logits,name="probabilities")

        predictions={
            "probabilities":y_proba,
            "labels":tf.cast(tf.round(y_proba),tf.int32,name="labels")
        }
        print(" << predictions >> ",predictions)

        #Define cost Function
        cost=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf_y,logits=logits),name="cost")

        #Define Optimizer
        optimizer=tf.train.AdamOptimizer(self.learning_rate)
        train_op=optimizer.minimize(cost,name="train_op")

    def train(self,X_train,y_train,num_epochs):
        with tf.Session(graph=self.g) as sess:
            sess.run(self.init_op)
            iteration=1
            for epochs in range(num_epochs):
                state=sess.run(self.initial_state)

                for batch_x,batch_y in create_batch_generator(X_train,y_train,self.batch_size):
                    feed={"tf_x:0":batch_x,
                          "tf_y:0":batch_y,
                          "tf_keepprob:0":0.5,
                          self.initial_state:state}
                    loss,_,state=sess.run(["cost:0","train_op",self.final_state],feed_dict=feed)

                    if iteration%20==0:
                        print("Epoch : %d/%d Iteration : %d  | Train Loss : %7.5f"%(epochs+1,num_epochs,iteration,loss))

                    iteration+=1
                if(epochs%10==0):
                    self.saver.save(sess,"model/sentiment-%d.ckpt"%epochs)

    def predict(self,X_data,return_prob=False):
        preds=[]
        with tf.Session(graph=self.g) as sess:
            self.saver.restore(sess,tf.train.latest_checkpoint("./model/"))
            test_state=sess.run(self.initial_state)
            for ii,batch_x in enumerate(create_batch_generator(X_data,None,batch_size=self.batch_size),1):
                feed={"tf_x:0":batch_x,
                      "tf_keepprob:0":1.0,
                      self.initial_state:test_state}
                if return_prob:
                    pred,test_state=sess.run(["probabilities:0",self.final_state],feed_dict=feed)
                else:
                    pred,test_state=sess.run(["labels:0",self.final_state],feed_dict=feed)
                preds.append(pred)

        return np.concatenate(preds)
    

In [0]:
import tensorflow as tf
n_words=max(list(word_to_int.values()))+1
rnn=SentimentRNN(n_words=n_words,
                seq_len=sequence_length,
                embed_size=256,
                lstm_size=128,
                num_layers=1,
                batch_size=100,
                learning_rate=0.001)


 << initial state >>  (LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/zeros:0' shape=(100, 128) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/zeros_1:0' shape=(100, 128) dtype=float32>),)
 << lstm_output >>  Tensor("rnn/transpose_1:0", shape=(100, 200, 128), dtype=float32)
 << final state >>  (LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_3:0' shape=(100, 128) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_4:0' shape=(100, 128) dtype=float32>),)
 << logits >>  Tensor("logits_squeezed:0", shape=(100,), dtype=float32)
 << predictions >>  {'probabilities': <tf.Tensor 'probabilities:0' shape=(100,) dtype=float32>, 'labels': <tf.Tensor 'labels:0' shape=(100,) dtype=int32>}
Instructions for updating:
Use tf.cast instead.


In [0]:
rnn.train(X_train,y_train,num_epochs=10)

Epoch : 1/10 Iteration : 20  | Train Loss : 0.00024
Epoch : 1/10 Iteration : 40  | Train Loss : 0.00009
Epoch : 1/10 Iteration : 60  | Train Loss : 0.00006
Epoch : 1/10 Iteration : 80  | Train Loss : 0.00005
Epoch : 1/10 Iteration : 100  | Train Loss : 0.00004
Epoch : 1/10 Iteration : 120  | Train Loss : 0.00003
Epoch : 1/10 Iteration : 140  | Train Loss : 1.71319
Epoch : 1/10 Iteration : 160  | Train Loss : 0.03230
Epoch : 1/10 Iteration : 180  | Train Loss : 0.01159
Epoch : 1/10 Iteration : 200  | Train Loss : 0.01352
Epoch : 1/10 Iteration : 220  | Train Loss : 0.01306
Epoch : 1/10 Iteration : 240  | Train Loss : 0.01417
Epoch : 2/10 Iteration : 260  | Train Loss : 3.49436
Epoch : 2/10 Iteration : 280  | Train Loss : 0.99158
Epoch : 2/10 Iteration : 300  | Train Loss : 0.34082
Epoch : 2/10 Iteration : 320  | Train Loss : 0.14943
Epoch : 2/10 Iteration : 340  | Train Loss : 0.08856
Epoch : 2/10 Iteration : 360  | Train Loss : 0.06205
Epoch : 2/10 Iteration : 380  | Train Loss : 2.973

In [0]:
pred=rnn.predict(X_test)
y_true=y_test[:len(pred)]
print("Test Accuracy : %.3f"%(np.sum(y_true==pred)/len(y_true)))

prob=rnn.predict(X_test,return_prob=True)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./model/sentiment-0.ckpt
Test Accuracy : 0.500
INFO:tensorflow:Restoring parameters from ./model/sentiment-0.ckpt
